In [72]:
import pandas as pd
import numpy as np
import pandas as pd
import glob
import re
from os.path import basename, splitext
import pickle
import matplotlib.pyplot as plt
import sys
sys.path.append('./')
import spacy
from labov import datasets, classifier, evaluation
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.metrics import classification_report, accuracy_score
from sklearn.utils import shuffle
from collections import Counter

In [2]:
enlp = spacy.load('en')
inlp = spacy.load('it')

In [3]:
it_train = pd.read_csv('../data/it_training.tsv', sep='\t')
it_test = pd.read_csv('../data/it_testing.tsv', sep='\t')
en_train = pd.read_csv('../data/en_training.tsv', sep='\t')
en_test = pd.read_csv('../data/en_testing.tsv', sep='\t')

wordCount = Counter(pd.concat([en_train.text,
                               it_train.text]).str.cat(sep='\n').split(' '))

In [78]:
wordCount['.']

60

In [4]:
def shaper(word):
    out = []
    for char in word.text:
        if char.isupper():
            if char in 'AEIOUY':
                out.append('V')
            elif char in 'BCDFGHJKLMNPQRSTVXZ':
                out.append('C')
            else:
                out.append(char)
        elif char.islower():
            if char in 'aeiouy':
                out.append('v')
            elif char in 'BCDFGHJKLMNPQRSTVXZ'.lower():
                out.append('c')
            else:
                out.append(char)
        else:
            out.append(char)
    return ''.join(out)

def bleach(text, word_dic=None):
    bleached = ' '.join([' '.join((shaper(w), 
                                   str(0)+str(len(w)),
                                   str(w.is_alpha),
                                   str(word_dic[w.text]))) for w in text])
    return bleached

bleach(inlp('this TeXt!!!'), wordCount)

'ccvc 04 True 263 CvCc 04 True 0 ! 01 False 36 ! 01 False 36 ! 01 False 36'

In [47]:
textit = (row.text for row in it_train.itertuples())
it_train['spacy'] = [doc for doc in inlp.pipe(textit,n_threads=4, batch_size=1000)]
it_train['bleached'] = it_train.spacy.apply(lambda x: bleach(x, wordCount))
texten = (row.text for row in en_train.itertuples())
en_train['spacy'] = [doc for doc in enlp.pipe(texten,n_threads=4, batch_size=1000)]
en_train['bleached'] = en_train.spacy.apply(lambda x: bleach(x, wordCount))

In [6]:
#it
textittest = (row.text for row in it_test.itertuples())
it_test['spacy'] = [doc for doc in inlp.pipe(textittest,n_threads=4, batch_size=1000)]
it_test['bleached'] = it_test.spacy.apply(lambda x: bleach(x, wordCount))
#en
textentest = (row.text for row in en_test.itertuples())
en_test['spacy'] = [doc for doc in enlp.pipe(textentest,n_threads=4, batch_size=1000)]
en_test['bleached'] = en_test.spacy.apply(lambda x: bleach(x, wordCount))

In [ ]:
#scores_m = cross_val_score(classifier.ngram, en_train.text, en_train.misogynous, cv=10, scoring='f1_macro')
#scores_mc = cross_val_score(classifier.ngram, en_train.text, en_train.misogyny_category, cv=10, scoring='f1_macro')
#scores_t = cross_val_score(classifier.ngram, en_train.text, en_train.target, cv=10, scoring='f1_macro')

In [ ]:
print(scores_m.mean(), scores_m.std())
print(scores_mc.mean(), scores_mc.std())
print(scores_t.mean(), scores_t.std())

In [ ]:
#bleached
scores_m_blc = cross_val_score(classifier.ngram, it_train.bleached, it_train.misogynous, cv=10, scoring='f1_macro')
scores_mc_blc = cross_val_score(classifier.ngram, it_train.bleached, it_train.misogyny_category, cv=10, scoring='f1_macro')
scores_t_blc = cross_val_score(classifier.ngram, it_train.bleached, it_train.target, cv=10, scoring='f1_macro')

In [71]:
print(scores_m_blc.mean(), scores_m_blc.std())
print(scores_mc_blc.mean(), scores_mc_blc.std())
print(scores_t_blc.mean(), scores_t_blc.std())

0.6884750342107324 0.09289533228840163
0.27440386625463287 0.051746507729429715
0.5753622173829789 0.07871571350648493


## Italian to English

In [54]:
m = classifier.ngram.fit(it_train.text, it_train.misogynous)
predictions_m = m.predict(en_test.text)

In [55]:
accuracy_score(en_test.misogynous,predictions_m)

0.508

## English to Italian

In [56]:
m = classifier.ngram.fit(en_train.text, en_train.misogynous)
predictions_m = m.predict(it_test.text)

In [57]:
accuracy_score(it_test.misogynous,predictions_m)

0.468

## Combined

In [66]:
ensample = en_train.sample(frac=1)
itsample = it_train.sample(frac=1)
combi = pd.concat([ensample, itsample])

In [68]:
m = classifier.ngram.fit(combi.bleached, combi.misogynous)
predictions_m = m.predict(en_test.bleached)
accuracy_score(en_test.misogynous,predictions_m)

0.583